# TransformCSVtoJSON
This notebook from pattern from some CSV file to JSON for visualization

In [ ]:
%matplotlib inline
from operator import add
from datetime import datetime, date
import calendar
import time, logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.patches as patches
import matplotlib.path as path
import json
import operator


In [ ]:
sc.stop() # We don't use spark here

# Functions

In [ ]:
#Loading Event Explanation for Converting Letter, for displaying 
cols=["Conc", "Letter", "MDLevent", "Refined Names"]
EventsDef=pd.read_csv('../EventDefExplanation.csv',delimiter=';',skiprows=1,names=cols)

print len(EventsDef)

conversionDict = {}
for i in range(len(EventsDef)):
    conversionDict[EventsDef["Conc"][i]]=EventsDef["Refined Names"][i]

#Add Churning Event
conversionDict["ChurningEvent"] = "ChurningEvent"
print conversionDict["ATM_Login"]

# Load

Here we can import from 2 differents 

In [ ]:
inputFile = "patternsMinimumSupportAllSegment0.005.csv"
inputFolder = "../results/ngram/"
csvPd = pd.read_csv(inputFolder + inputFile)
Segment = ""
valueKey = "nbrChurning"
onlyEndWithCE = True

In [ ]:
inputFile = "AllCluster_Suggested_Patterns.xlsx"
inputFolder = "../selected/"
csvPd = pd.read_excel(inputFolder + inputFile)
Segment = "" # Cluster2 Cluster4 Cluster6 Cluster10 Cluster12
valueKey = "1_sum(custCount)0.0"
onlyEndWithCE = False

In [ ]:
if not Segment == "":
    csvPd = csvPd[(csvPd.Segment == Segment) ]

In [ ]:
csvPd.iterrows().next()[1]

In [ ]:
linkDict = {}
maxDeep = 7

for row in csvPd.iterrows():
    value = row[1][valueKey]
    sequence = row[1]['pattern'][2:][:-2].split("', '")
    
    # Limit patterns lenght
    if len(sequence) > maxDeep:
        if onlyEndWithCE:
            sequence = sequence[len(sequence)-maxDeep:]
        else:
            sequence = sequence[:-(len(sequence)-maxDeep)]
    
    # Only get churning events:
    if onlyEndWithCE:
        if not sequence[-1] == "ChurningEvent":
            continue
        sequence = list(reversed(sequence))
    
    # Loop init
    event = sequence[0] # last event / first event
    d = 0
    #deep = len(sequence) - d
    if onlyEndWithCE:
        eventDeep = event + ' ' + str(d)
    else:
        eventDeep = event + ' ' + str( - d + maxDeep)
    source = eventDeep
    
    for event in sequence[1:]:
        d = d + 1
        #deep = len(sequence) - d
        if onlyEndWithCE:
            eventDeep = event + ' ' + str(d)
        else:
            eventDeep = event + ' ' + str( - d + maxDeep)
        target = eventDeep#eventDict[eventDeep]
        
        if not source == target: # D3 doesn't allow
            key = (source, target)
            if key not in linkDict:
                linkDict[key] = value
            else:
                linkDict[key] = linkDict[key] + value

        source = target
        

In [ ]:
# Max value
maxValue = 0
for source, target in linkDict:
    value = linkDict[(source, target)]
    if value > maxValue:
        maxValue = value
        
print maxValue

minValue = maxValue * 0.005 # 1000  # Take 1% (or 0.5% of the stuff)
print minValue

In [ ]:
# Max/min value, based on the maximun number of links
maxLink = maxDeep * 20
valueList = linkDict.values()
valueList.sort(reverse=True)
maxLink = min(maxLink, len(valueList)-1)
minValue = valueList[maxLink]

In [ ]:
print "Number linkDict: " + str(len(linkDict))

links = []
eventDict = {}
for source, target in linkDict:
    value = linkDict[(source, target)]
    if value < minValue:
        continue
        
    if source not in eventDict:
        eventDict[source] = len(eventDict)
    if target not in eventDict:
        eventDict[target] = len(eventDict)
    
    #Source and Target switched for chronology:
    if onlyEndWithCE:
        links.append({'source': eventDict[target], 'target': eventDict[source], 'value': value})
    else:
        links.append({'source': eventDict[source], 'target': eventDict[target], 'value': value})
    
print "Number links: " + str(len(links))

In [ ]:
print len(eventDict)

In [ ]:
eventDictSorted = sorted(eventDict.items(), key=operator.itemgetter(1))

In [ ]:
nodes = []
for event, i in eventDictSorted:
    depth = int(event[-1:]) # Should 1 digit be careful. Used to x position of the events, for chronology
    event = event[:-2] # Remove the depth number
    event = conversionDict[event]
    nodes.append({'name': event, 'id': str(i), 'depth': depth})

In [ ]:
nodes

# Saving

In [ ]:
print "Number of nodes: " + str(len(nodes))
print "Number of links: " + str(len(links))

In [ ]:
finalJson = {}
finalJson['nodes'] = nodes
finalJson['links'] = links

outputFilePath = 'input/'+inputFile+'_'+Segment+'.json'
with open(outputFilePath, 'w') as outfile:
    json.dump(finalJson, outfile)

print outputFilePath